<a href="https://colab.research.google.com/github/MfmRifath/LLM_Learning/blob/main/meeting_minutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q requests torch bitsandbytes sentencepiece accelerate openai transformers

In [2]:
import os
import requests
from IPython.display import Markdown, display
from openai import OpenAI
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
from google.colab import drive

In [3]:
AUDIO_MODEL="whisper-1"
LLAMA='meta-llama/Meta-Llama-3.1-8B-Instruct'

In [4]:
drive.mount("/content/drive")
audio_filename ="/content/drive/MyDrive/LLM/denver.mp3"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [6]:
open_api_key=userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=open_api_key)

In [7]:
audio_file=open(audio_filename, "rb")
transcription=openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format='text')
print(transcription)

Thank you. Um. Okay. Thank you. This is a meeting of the arm of Springfield for July the 16th of 2024 at exactly six p.m. calling the meeting to order. Um we will do introduction here. I'm mayor of the city of Springfield. I'm also the CEO was at the controls. Our CEO calling Draper is present. Um deputy mayor to my right in the descending order and counselor Ward. One is Glenn fuel. Counselor war. Two is Andy Kaczynski. Counselor Ward. Three is Mark Miller. Counselor Word four is Melinda Warren. Um Melinda, if you want to do invocation. Maybe I'll enter your name. I'm Melinda Warren, and I'm here for the betterment of our municipality. Thank you very much. Um then I'll do the land acknowledgement. The arm of Springfield of knowledge is that we are gathered on ancestral lands. Treaty one territory, traditional territory, the initial Abby Cree O. G. Cree Dakota DNA people on the national homeland and on the national homeland of the Red River, Métis. Um that is item number three and item

In [8]:
system_message='You are an assintent that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown'
user_prompt=f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including summery with attendees, location and date; discussion points; takeaways; and an action items with owner's {transcription} "
messages =[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
]

In [9]:
quant_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, quantization_config=quant_config, device_map="auto")
outputs = model.generate(inputs=inputs, streamer=streamer, max_new_tokens=2048, use_cache=True)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assintent that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including summery with attendees, location and date; discussion points; takeaways; and an action items with owner's Thank you. Um. Okay. Thank you. This is a meeting of the arm of Springfield for July the 16th of 2024 at exactly six p.m. calling the meeting to order. Um we will do introduction here. I'm mayor of the city of Springfield. I'm also the CEO was at the controls. Our CEO calling Draper is present. Um deputy mayor to my right in the descending order and counselor Ward. One is Glenn fuel. Counselor war. Two is Andy Kaczynski. Counselor Ward. Three is Mar

In [11]:
response = tokenizer.decode(outputs[0])


In [12]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assintent that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including summery with attendees, location and date; discussion points; takeaways; and an action items with owner's Thank you. Um. Okay. Thank you. This is a meeting of the arm of Springfield for July the 16th of 2024 at exactly six p.m. calling the meeting to order. Um we will do introduction here. I'm mayor of the city of Springfield. I'm also the CEO was at the controls. Our CEO calling Draper is present. Um deputy mayor to my right in the descending order and counselor Ward. One is Glenn fuel. Counselor war. Two is Andy Kaczynski. Counselor Ward. Three is Mark Miller. Counselor Word four is Melinda Warren. Um Melinda, if you want to do invocation. Maybe I'll enter your name. I'm Melinda Warren, and I'm here for the betterment of our municipality. Thank you very much. Um then I'll do the land acknowledgement. The arm of Springfield of knowledge is that we are gathered on ancestral lands. Treaty one territory, traditional territory, the initial Abby Cree O. G. Cree Dakota DNA people on the national homeland and on the national homeland of the Red River, Métis. Um that is item number three and item number four. Thank you very much. Melinda and Glenn. Any questions, additions or modifications to the agenda? Mark. Close meeting. Okay? Anything else? I see none. All right. That is carried. Um mover and a seconder for the adoption of the minutes, Patrick and, uh, Andy. Any questions additions from council adoption of the minutes for July the second. I see none show hands those in support and it's carried. We have a question period. Nobody's in front of us here. We have 15 minutes allocated. The only question period. I see nobody online as well. So at this point here will carry on to consent agenda. That's item number 10. Um oh, sorry. Um Tammy, did you have anything for a question period at all? I just saw you coming in. Okay Uh, then what we'll do is we'll do the consent agenda for the first two and 10.3, so I'm not sure we can actually do the consent agenda there. So if I can do, uh, the first 110.1 the A. M. M news bulletin kind of mover and a seconder for that, please. Mark and Glenn. So any, uh, questions with regards to that from council. I see none. I can get a show hands of those who have questions for the consent agenda period. The next one is a 10.2. That's stars. Can I get a mover in a sector for that, Patrick and Glenn? Um I had a comment here there. I am a big proponent for stars. This is a vital component for health care. I've had first firsthand experience with stars and I've had a lot of experience with air ambulance. Uh and the north rely on air ambulance provided by the government to either via airport, helicopter. Um and air ambulance propeller or jet. Um when there's babies involved in adults like this is extremely good. Um uh, tool for the health department. It has been utilized in the arm Springfield. Um and that's all I had to say with regards to That is unanimous and is carried. Then if we can get to the thank you card of the SCI grad recipients and mover in a sector for 10.3, please. Melinda and Andy. Um, at this point here, we have the card from Layla plant there. We increase the amount to $750 from 500, which I think is was an exceptional idea from our deputy mayor to increase that because most of the honorary degrees and so on are at that 750 level. Um and I'm not saying that we have to match costs there, but it was really well well received. Uh the SCI staff principal Kevin Dell extremely large crowd present as every year on their families at the club region Casino, which is an impressive facility, but still not in our community. So it'll be nice to have the grad eventually be here at the our new recreational facility, which requires our new water treatment plant to be able to operate and provide our community going forward. So appreciate the card from from Layla is very well done. Um any other questions or comments from council at all? I see none. Can I get a show hands of those in support of 10.3? Thank you card. Unanimous and is carried. We'll get to a new business there, and that's the purchase of the asphalt pot box trailer mover and a seconder for that, please. Linda and Andy. Any questions are from council with regards to this. You have your hand up, honey. Yes, Mr Mayor. I would like to ask if we purchasing ourselves or we using that, uh, company that helping purchase equipment through a man. Can you? Yeah. They can help us with purchasing equipment. We never use them like purchasing the trucks or a fire department chassis and Police car. I believe the director of public works is online, but I honestly don't think that the industrial machine Inc is part of that canoe procurement. They have to be a listed supplier. Um, so maybe blame if you're able to confirm if that's an opportunity through canoe procurement or not. Yeah, we did look into seeing if that was listed company, and I don't believe they were so. I'm not 100% sure if there is any advantages for the municipality for canoe in this equipment purchase. So. Okay. Thank you. Any other questions from council at all? I see none. Um, then I can read the resolution. It was all the council of the arm of Springfield approves the purchase of one new asphalt hot box trailer from industrial machine Inc. Not to exceed the budgeted amount of $155,000, including taxes with funds coming from the vehicle and equipment reserve. I got to show hands of the first in support of purchase of asphalt hot box trailer. That's Melinda. That's unanimous, and it is carried. Then we'll get to 11.2 purchase of a rehab trailer or specifically a rehabilitation trailer for the prior department moving a second for that place. Patrick and Glenn. Be resolved. The council of the arm of Springfield approves the purchase of one new rehabilitation trailer from pro pack for $59,630.27 US plus applicable taxes, transport fees and broker fees with funds to come from the vehicle and equipment reserve. Are we able to amend the resolution to show Canadian funds beside it there? I think it's $80,000. Um, it's 59,000 637 20 U. S. But when we write the check, we have to get the exact US amount on that day of the check writing. Oh, so that will change. It could change from what's being proposed right now. Canadian so Okay. All right. I understand that. Can I get a show any questions from council with regards to that? I see none going to get a show hands those in support of the purchase rehabilitation trailer. That is unanimous, and it is carried 11.3. That's financial statements. Mover and seconder please. Melinda Mark. I had one question with regards to financial statements. It says here basically regards to notes to financial statements of May 31 24 the tax living grants and Lou have been collected, which accounts for our deficit. Uh can we have some clarification on this? So there's there isn't there's no misinformation with regards to that. So we're still waiting for the tax levy and the grants to come through. Are they delayed for any reasons? Okay, so this is routine, then it's not different to this year. Okay. Any other questions from council at all? I see none. I see none. Then can I get a show hands of financial statements? Uh, those in support. That is unanimous, and it is carried. 11.4 list of accounts a mover and a seconder for that place. That's a Melinda and Glenn. Whereas disbursements have been reviewed for the period of June 7th to July 5th 2024 be it resolved that all accounts listed on the attached print out from June 7th 2024. 2442 to 160721 totaling 2,000,088,000 127 56 be approved for payment as well as the following E F T entries. Western Financial 15,000 127 68 Amelia 3 58 35 Superpass 1 77 59 John Deere 8000 4 48 91 6,005 76 83 6,005 76 83 and be it further resolved that June payroll in the amount of 3 73 5 77 33 be approved. Any questions, comments from council at all. Mark. Thank you, Mr Mayor. Thank you, Mr Mayor. Thank you, Mr Mayor. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Any other questions come from council already. Mark. Yeah, we collect points for all our purchases. Yeah. The questions are from council. All right. You're on the phone. Yes, sir. Hello. This is the location support to the list of accounts under 11.4. It was a unanimous and scary. If we can get to 11.5, that's a nonprofit tax. Cancellation, a centralized performing arts. Center of excellence or space. A mover on a second for that. Please Patrick and Melinda. Dears. I have a question about that. The taxes are 69 57. Plus the arrears. So that would take us to a ballpark of 7,100. But we're looking at $7,900 for that. I'm sure what the math is there. So the tax bill that council is seeing a copy of. Those arrears are not to date. So that was when. The original bill was printed in July of 2023. Oh, I see. So the 140 is still standing. So that's 140. Okay. There's monthly penalties that have been accruing since that time. Oh, so that's where the 79 is from. Yeah. Okay. Any questions. Yes, I would like to ask if they always paying penalties, they never come up with that, or this is just the first time. Yeah. So they were kind of in a strange situation where they were also renting additional space in the mall. And owned this property as well. So they had come to council for a council cancellation of the property taxes at that time. Council didn't agree with the cancellation because the activities weren't. Taking place in the building on main street. Now that they've. You know, gotten rid of their space in the mall, They've come back to council and now it's council's opportunity to approve or deny again. I know, but that's not what I asked. I asked you if they always on our ears. No, they're not always in a rare. Okay. No, this is the first time. Okay. Thank you. Yeah. And as a council, we've had discussions about this. In camera. And out of camera there. It's it's to the point. There'll be no further. Assistance in my opinion. I know it's a, it's a. It's a one vote there, but I think. Hopefully this, this helps there, but. We have to. Take that into consideration there. Any other comments from council. I see none. Can I get a show of hands of those in support of the nonprofit taxation cancellation space? Okay. I see six letter of support for a John Q built for a collaborative efficiency advocate. Position. Can I get a mover in a second with regards to that place? Melinda and Glenn. Any questions from a council with regards to a letter of. Of support. Andy. Yeah. I think the question that we use this. Efficiency money. We can, we can go directly ourselves to efficiency. We don't have to go through John Q. No, but this is John Q asking that they're going to go align with. Efficiency Manitoba. They've got a grant for 120,000. And the John Q will likely match that to a certain level. I don't know. If they're going to do that. But we're linking with efficiency Manitoba. The government entity there. To get that special environmental incentive as well. They have a 5%. MECD. I'm trying to remember what that acronym stands for, but it'll. The, the structure, our John Q daycares are built. Better than the Manitoba environmental. I'm trying to remember what that acronym stands for. I'm trying to remember if that's what MECD stands for. Or something with regards to that. So. Efficiency Manitoba and John Q with their daycare program. Is, is better than the, what the, the government is, is looking for. And efficiency Manitoba will work with us to, to do even a better job. And this is a. Aligning directly with them for, for John Q. So, yeah, they, anybody can, you can, can, as a small business and so on, use them. But John Q is, has been using them and now they're. Looking to, to have a. Direct contact with them to, to make the initiatives that much better. And just asking for support with regards to the green initiatives. That efficiency Manitoba will, will give to them as well. Because we're a part of that group of. I want to take metropolitan region there. They're, they're throwing it out to surrounding areas. City, Winnipeg. Understand, but. We kind of question what John Q did for us over the years. And they didn't do absolutely nothing till now. So we paying the. The bill for, for them. I don't know, $20,000 a year. What do we get in the return? Well, we're, we're, I don't know if it's, it's a one-time payment. What I understand is a $20,000. There. We're getting like daycare that 39 acres of land that we. We bought there, which was a very prudent. I think it's a $50,000 part there. Because our land is, is only going up in value, but we have no land for a daycare. This community needs a daycare. As a high priority. For allowing us to go forward there because daycare is, is huge and just being a grandparent and with my, with my sons. And. And my grandkids. It's a huge cost. And location, location, location. Yeah. Like there's a lot of daycare like they have in East Selkirk. Loretta. In the city of Winnipeg, there's 20 plus. These daycares around. There they're very efficient. There's actually a university. I know the name, but I'm not going to mention the name. I'm not going to mention the name of the university, but there's a lot of. Insight from, from other. Entities there and the, the energy efficiency. Coming from these buildings. The costs. Our streamline. And not to mention the environmental green initiative is, is a top. So we are. We are getting lots of from this. And our water treatment plant. And once we get everything ready, our water treatment plant. Running there. To allow our community to go forward and daycare is, is something that this community needs. Nobody say that we don't need that, but. Not necessarily the land will be purchased. We have land that we want to build. So, you know, we could have, you know, Absorb an amount of money for that land and we could have used that one for that purpose too. So. It's not only judges because we purchased the land. Fits, you know, the bill for, for that daycare center. We know what they say that we don't need the daycare center. You know, We, we have, we, we had the land to, to pull the daycare center. So, you know, I think that's a good point. Thank you. Thank you. Thank you. So at all. Then if I can get a vote. Or can I get the resolution? Be it resolved that the mayor be authorized to sign a letter of support for JQ built for a collaborative advocate position on behalf of the arm of Springfield. I got to show hands of those in support of a letter of support for John Q. That is a Melinda Glenn and Patrick. Those post. Mark and Andy. 11.7 amount of municipalities administrators review and revision. 11.7 a mover and a seconder for that. Please. Melinda and Patrick. Any questions from council with regards to that. I can get the resolution in red. Did you have a question? Who structured this letter to us. Municipality of administrators. Correct. Yeah. What, what can you tell us about this organization? Yeah, so. The MMA. So Manitoba municipal administrators is kind of my AMM to council. So it's the organization I belong to, and I have to get my professional credits through to keep a good standing with them. So it's this group that was asked to put. There, you know, comments. And they put together a resolution and sent it out to all the members and asked that we bring it forward to council for review. And if you want to vote on the resolution and provide it. That's an option. So. So. They stayed in here. Municipality and wasteful processes and their interaction, I don't know. I don't know. I don't know. Oh, well, I think the province is doing a review of the Manitoba municipal board mandate. So they've reached out to certain groups to comment. And they reached out to MMA to comment on this. Yeah, but this is kind of something to do with the bill 37. I don't know specifically. But I can confirm, but. The municipal board follows falls under a different act. So. This is specific. To the municipal board. Okay. Thank you. Any other questions council. Have you read the resolution yet? No. Okay. Do we have to read it? Allowed, or can we agree to that? We read the therefore. Be it resolved section. Therefore be it resolved that the AMM lobby, the province of Manitoba to undertake a comprehensive review of the mandate role and function. Municipal board. And further that the province of Manitoba complete a third party value for money service delivery review of the municipal board processes and undertake process improvements. To streamline functions, reduce red tape and reduce municipal costs. And further that the province of Manitoba engage AMM and MMA to participate as key stakeholders in the preparation of terms of reference for these efforts. And any steering or oversight body for this work. With the resolution. I read the, can I get a show of hands? Are those in support? The match of a municipal administrators review and revision. That is unanimous and is carried. 11.8 to council schedule schedule. Can I get a mover and a seconder? Patrick and Melinda. Be it resolved that the following changes be approved to the council meeting schedule Tuesday, October 8th committee of the whole canceled. Any questions council. I see none. And why are we counseling for what reason? Council just approved the planning session with council and staff. On October 7th to nine. Okay, thank you. I got a show of hands. Those in support of the council schedule. 11.8. That's unanimous. And sorry, Andy, do you have your hand up? Yeah, that's unanimous. Okay. So we're prepared to close. The, the meeting. For in-camera there. Can I get a mover and a seconder for that, please? Melinda and Patrick. Be it resolved that this meeting recessed to in-camera to discuss legal issues. Okay. Be it resolved that all matters shall remain confidential until report is made public. Okay. Thank you very much. It's a six 57 PM. We're coming out of a closed meeting. Can I get a mover and a second or an error to come out of the close meeting? Okay. We've got nothing more on the agenda. Everything was discussed in camera. If we can. Adjourn the meeting and mover and a seconder for that, please. Melinda Patrick. Then the meeting is adjourned at six. Oh, show of hands. Okay.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of the Arm of Springfield Council Meeting**
**Date:** July 16, 2024
**Location:** Arm of Springfield Council Chamber
**Attendees:**

* Mayor
* Deputy Mayor
* Councillor Ward 1 (Glenn Fuel)
* Councillor Ward 2 (Andy Kaczynski)
* Councillor Ward 3 (Mark Miller)
* Councillor Ward 4 (Melinda Warren)

**Summary:**

The Arm of Springfield Council meeting was held on July 16, 2024, at 6:00 PM. The meeting began with introductions, land acknowledgement, and a review of the agenda. The council then proceeded to discuss various items, including the consent agenda, the purchase of an asphalt hot box trailer, the purchase of a rehabilitation trailer, financial statements, and a list of accounts. The council also discussed a letter of support for JQ Built for a Collaborative Advocate Position and a resolution regarding the Manitoba Municipal Administrators (MMA) review and revision of the municipal board mandate.

**Key Discussion Points:**

* The consent agenda was discussed, with the council approving items 10.1 and 10.2.
* The council approved the purchase of an asphalt hot box trailer from Industrial Machine Inc. for $155,000, including taxes.
* The council approved the purchase of a rehabilitation trailer from Pro Pack for $59,630.27 US, including taxes, transport fees, and broker fees.
* The council discussed financial statements, with Councillor Mark Miller asking for clarification on the tax levy and grants.
* The council approved the list of accounts, including disbursements and EFT entries.
* The council discussed a letter of support for JQ Built for a Collaborative Advocate Position and approved the resolution.
* The council approved the MMA resolution regarding the review and revision of the municipal board mandate.

**Takeaways:**

* The council approved several items on the consent agenda.
* The council approved the purchase of an asphalt hot box trailer and a rehabilitation trailer.
* The council discussed financial statements and approved the list of accounts.
* The council approved a letter of support for JQ Built for a Collaborative Advocate Position.
* The council approved the MMA resolution regarding the review and revision of the municipal board mandate.

**Action Items:**

1. **Purchase of Asphalt Hot Box Trailer:** The council approved the purchase of an asphalt hot box trailer from Industrial Machine Inc. for $155,000, including taxes.
2. **Purchase of Rehabilitation Trailer:** The council approved the purchase of a rehabilitation trailer from Pro Pack for $59,630.27 US, including taxes, transport fees, and broker fees.
3. **Letter of Support for JQ Built:** The council approved the letter of support for JQ Built for a Collaborative Advocate Position.
4. **MMA Resolution:** The council approved the MMA resolution regarding the review and revision of the municipal board mandate.

**Owner's:**

1. **Mayor:** Mayor is responsible for signing the letter of support for JQ Built and the MMA resolution.
2. **Deputy Mayor:** Deputy Mayor is responsible for reviewing the MMA resolution and providing input.
3. **Councillor Ward 1 (Glenn Fuel):** Councillor Fuel is responsible for reviewing the consent agenda and providing input.
4. **Councillor Ward 2 (Andy Kaczynski):** Councillor Kaczynski is responsible for reviewing the financial statements and providing input.
5. **Councillor Ward 3 (Mark Miller):** Councillor Miller is responsible for reviewing the list of accounts and providing input.
6. **Councillor Ward 4 (Melinda Warren):** Councillor Warren is responsible for reviewing the MMA resolution and providing input.<|eot_id|>